In [ ]:
import pandas as pd
MAIN_DATA = pd.read_csv("/content/warszawa.csv")

df_warsaw = MAIN_DATA.copy()
df_warsaw.head()

!pip install spacy -U
!python -m spacy download pl_core_news_lg

import spacy
nlp = spacy.load("pl_core_news_lg")
nlp.pipeline

import re
import regex as re

Część 1 - przetwarzanie i oczyszczanie danych


In [3]:
#Zamień skróty nazw dni tygodnia na polskie odpowiedniki (pełne nazwy dni tygodnia) - kolumna created_at
df_warsaw.created_at = [row.replace("Mon", "poniedziałek").replace("Tue", "wtorek").replace("Wed", "środa").replace("Thu", "czwartek").replace("Fri", "piątek").replace("Sat", "sobota").replace("Sun", "niedziela") for row in df_warsaw ["created_at"]]

In [5]:
#Zamień skróty nazw miesięcy na liczbowe odpowiedniki (np Jun na 06) - kolumna user_created_at
df_warsaw.user_created_at = [row.replace("Jan", "01").replace("Feb", "02").replace("Mar", "03").replace("Apr", "04").replace("May", "05").replace("Jun", "06").replace("Jul", "07").replace("Aug", "08").replace("Sep", "09").replace("Oct", "10").replace("Nov", "11").replace("Dec", "12") for row in df_warsaw ["user_created_at"]]

In [ ]:
#Pobierz wszystkie linki do tweetów i przekaż je do listy (tweet_url)

z = -1
tekst =""
for el in df_warsaw:
  z = z+1
  doc = nlp(df_warsaw.tweet_url[z])
  for el in doc.ents:
    tekst = tekst + " " + el.text
  
linki_do_tweetów = re.findall("\S+", tekst)

print(linki_do_tweetów)

In [ ]:
#Pobierz wszystkie linki znajdujące się w tweetach i przekaż je do listy (kolumna urls)

z = -1
tekst = ""
for el in df_warsaw:
  z = z+1
  if type(df_warsaw.urls[z]) == str:
    tekst = tekst + " " + df_warsaw.urls[z]


doc = nlp(tekst)
for el in doc.ents:
  x = x + " " + el.text

linki_w_tweetach = re.findall("\S+", x)

print(linki_w_tweetach)

In [ ]:
#Pobierz wszystkie linki do obrazków i przekaż je do listy (kolumna media)

z = -1
tekst = ""
for el in df_warsaw:
  z = z+1
  if type(df_warsaw.media[z]) == str:
    tekst = tekst + " " + df_warsaw.media[z]

doc = nlp(tekst)
for el in doc.ents:
  x = x + " " + el.text

linki_do_obrazków = re.findall("\S+", x)
print(linki_do_obrazków)

In [ ]:
#Usuń wszystkie słowa oznaczone jako stopwords i przekaż oczyszczony tekst do nowej kolumny o nazwie text_without_stopwords

def clean_text(tekst):
  doc = nlp(tekst)
  for token in doc:
    x = token.text
    if token.is_stop == True:
      cleaned_text = tekst.replace(x, "")
      return cleaned_text

df_warsaw["text_without_stopwords"] = df_warsaw["text"].apply(clean_text)

print(df_warsaw["text_without_stopwords"])

Część 2 - eksploracyjna analiza danych

In [ ]:
#Wyświetl tylko zweryfikowanych użytkowników (kolumna user_verified)
for index, row in df_warsaw.iterrows():  
  if row['user_verified'] != "False":
    print (row['user_name'])

In [ ]:
#Wyświetl tylko te tweety, które nie są uznane jako “wrażliwe” (kolumna possibly_sensitive)

for index, row in df_warsaw.iterrows():  
  if row['possibly_sensitive'] != "True":
    print (row['text'])

In [ ]:
#Wypisz top5 tweetów z największa liczbą polubień.

top5_polubienia = df_warsaw.nlargest(5, 'favorite_count', keep='all')
print(top5_polubienia.text)

In [ ]:
#Wypisz top5 tweetów z największa liczbą retweetów.

top5_retweety = df_warsaw.nlargest(5, 'retweet_count', keep='all')
print(top5_retweety.text)

In [ ]:
#Wyświetl tweety użytkownika, który założył konto najwcześniej (spośród wszystkich użytkowników w datasecie) (kolumna user_created_at).

import datetime

df_warsaw['user_created_at'] = pd.to_datetime(df_warsaw['user_created_at'])

najwczesniej = df_warsaw.nsmallest(1, 'user_created_at', keep='all')

print(najwczesniej.text)

In [ ]:
#Wyświetl tweety użytkownika. który ma najwięcej followersów.

najwiecej = df_warsaw.nlargest(1, 'user_followers_count', keep='all')

print(najwiecej.text)

In [ ]:
#Podaj, w który dzień tygodnia najczęściej były publikowane tweety z datasetu

df_warsaw.created_at = [row.replace("poniedziałek", "1").replace("wtorek", "2").replace("środa", "3").replace("czwartek", "4").replace("piątek", "5").replace("sobota", "6").replace("niedziela", "7") for row in df_warsaw ["created_at"]]

pn = 0
wt = 0
sr = 0
cz = 0
pt = 0
sb = 0
nd = 0

for el in df_warsaw.created_at:
  if el[0] == "1":
    pn = pn + 1
  elif el[0] == "2":
    wt = wt + 1
  elif el[0] == "3":
    sr = sr + 1
  elif el[0] == "4":
    cz = cz + 1
  elif el[0] == "5":
    pt = pt + 1
  elif el[0] == "6":
    sb = sb + 1
  elif el[0] == "7":
    nd = nd + 1

print("poniedziałek: ", pn)
print("wtorek: ", wt)
print("środa: ", sr)
print("czwartek: ", cz)
print("piątek: ", pt)
print("sobota: ", sb)
print("niedziela: ", nd)

lista=[pn, wt, sr, cz, pt, sb, nd]

najwieksza = None

for i in lista:
  if najwieksza == None or najwieksza < i:
    najwieksza = i
if najwieksza == lista[0]:
  print("Najczęstszy dzień to jest poniedziałek")
if najwieksza == lista[1]:
  print("Najczęstszy dzień to jest wtorek")
if najwieksza == lista[2]:
  print("Najczęstszy dzień to jest środa")
if najwieksza == lista[3]:
  print("Najczęstszy dzień to jest czwartek")
if najwieksza == lista[4]:
  print("Najczęstszy dzień to jest piątek")
if najwieksza == lista[5]:
  print("Najczęstszy dzień to jest sobota")
if najwieksza == lista[6]:
  print("Najczęstszy dzień to jest niedziela")


Część 3 - przetwarzanie języka naturalnego

In [ ]:
#Rozpoznaj osoby (persName) i przekaż je do kolumny o nazwie persons
#Rozpoznaj miejsca (placeName) i przekaż je do kolumny o nazwie places
#Rozpoznaj organizacje (orgName) i przekaż je do kolumny o nazwie organizations

df_warsaw["persons"] = ""
df_warsaw["places"] = ""
df_warsaw["organizations"] = ""


n = -1

for el in df_warsaw:
  n = n+1
  doc = nlp(df_warsaw.text[n])
  for el in doc.ents:
    y = el.label_
    x = el.text
    if y == "persName":
      df_warsaw["persons"] = df_warsaw["persons"] + " " + x
    elif y == "placeName":
      df_warsaw["places"] = df_warsaw["places"] + " " + x
    elif y == "orgName":
      df_warsaw["organizations"] = df_warsaw["organizations"] + " " + x


print (df_warsaw["persons"])
print (df_warsaw["places"])
print (df_warsaw["organizations"])

Część 4 - rozwiązywanie problemów/umiejętność interpretowania dokumentacji


Wykorzystując pakiet matplotlib wykonaj wykres ilustrujący liczbę tweetów per dzień tygodnia



In [29]:
#Wykorzystując pakiet matplotlib wykonaj wykres ilustrujący liczbę tweetów per dzień tygodnia

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
lista=[pn, wt, sr, cz, pt, sb, nd]
bars = ('poniedziałek', 'wtorek', 'środa', 'czwartek', 'piątek', 'sobota', 'niedziela')
y_pos = np.arange(len(lista))
 
plt.figure(figsize=(10,5))
 
plt.bar(y_pos, lista, color = 'teal')

plt.xticks(y_pos, bars)
 
plt.xlabel('Dzień tygodnia', fontsize=12, color='darkslategray')
plt.ylabel('Liczba tweetów', fontsize=12, color='darkslategray')
plt.title('Liczba tweetów per dzień tygodnia', fontsize=16, color='black')
 
plt.show();